In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/elon-musks-tweets-dataset-2022/cleandata.csv
/kaggle/input/elon-musks-tweets-dataset-2022/rawdata.csv
/kaggle/input/elon-musks-tweets-dataset-2022/userinfo.csv


In [2]:
#import necessary libraries

import pandas as pd 
import numpy as np 
import re
from textblob import TextBlob

%matplotlib inline 
import matplotlib.pyplot as plt 
import seaborn as sns 

import warnings
warnings.filterwarnings("ignore")

In [3]:
#read the raw dataset

tweet=pd.read_csv("../input/elon-musks-tweets-dataset-2022/rawdata.csv")
tweet.head(10)

,Tweets,Retweets,Likes,Date
0,@PeterSchiff 🤣 thanks,209,7021,2022-10-27 16:17:39
1,@ZubyMusic Absolutely,755,26737,2022-10-27 13:19:25
2,Dear Twitter Advertisers https://t.co/GMwHmInPAS,55927,356623,2022-10-27 13:08:00
3,@BillyM2k 👻,802,19353,2022-10-27 02:32:48
4,Meeting a lot of cool people at Twitter today!,9366,195546,2022-10-26 21:39:32
5,Entering Twitter HQ – let that sink in! https:...,145520,1043592,2022-10-26 18:45:58
6,@dogeofficialceo @mtaibbi 🤣💪,194,3611,2022-10-26 17:18:30
7,@mtaibbi 🤣,117,2848,2022-10-26 17:14:07
8,@ARodTV Definitely closer to citizen journalis...,699,10189,2022-10-26 17:05:16
9,@sandyleevincent Nobody bats 1000 🤷‍♂️,126,2920,2022-10-26 15:42:50


First, we have to clean the tweets.
Looking deep into the "Tweets" column we can observe there is a use of a lot of emojis, mentions, links and many more. 

In [4]:
#create a user define function to clean the text

def clean_text(text):
    #first include all possible emoji in a variable
    emoji=re.compile("["
                     u"\U0001F600-\U0001F64F"    #emoticons
                     u"\U0001F300-\U0001F5FF"    #symbol & pictographs
                     u"\U0001F680-\U0001F6FF"    #transport & map
                     u"\U0001F1E0-\U0001F1FF"    #flags
                     u"\U00002500-\U00002BEF"    #chinese char
                     u"\U00002702-\U000027B0"
                     u"\U000024C2-\U0001F251"
                     u"\U0001f926-\U0001f937"
                     u"\U00010000-\U0010ffff"
                     u"\u2640-\u2642" 
                     u"\u2600-\u2B55"
                     u"\u200d"
                     u"\u23cf"
                     u"\u23e9"
                     u"\u231a"
                     u"\ufe0f"    # dingbats
                     u"\u3030"
                              "]+", re.UNICODE)
    text=re.sub('RT','',text)    #remove 'RT' from tweets
    text=re.sub('#[A-Za-z0-9]+','',text)    #remove hashtags from tweets
    text=re.sub('\\n','',text)    #remove \n from the tweets
    text=re.sub('https?:\/\/\S+','',text)    #remove the hyperlinks from the tweets
    text=re.sub('@[\S]*','',text)    #remove the mentions from the tweets
    text=re.sub('^[\s]+|[\s]+$','',text)    #remove leading & trailing whitespace from the tweets
    text=re.sub(emoji,'',text)    #remove the emojis from the tweets
    
    return text

In [5]:
#creating new column named CleanedTweets

tweet['CleanedTweets']=tweet['Tweets'].apply(clean_text)
tweet.head(10)

,Tweets,Retweets,Likes,Date,CleanedTweets
0,@PeterSchiff 🤣 thanks,209,7021,2022-10-27 16:17:39,thanks
1,@ZubyMusic Absolutely,755,26737,2022-10-27 13:19:25,Absolutely
2,Dear Twitter Advertisers https://t.co/GMwHmInPAS,55927,356623,2022-10-27 13:08:00,Dear Twitter Advertisers
3,@BillyM2k 👻,802,19353,2022-10-27 02:32:48,
4,Meeting a lot of cool people at Twitter today!,9366,195546,2022-10-26 21:39:32,Meeting a lot of cool people at Twitter today!
5,Entering Twitter HQ – let that sink in! https:...,145520,1043592,2022-10-26 18:45:58,Entering Twitter HQ – let that sink in!
6,@dogeofficialceo @mtaibbi 🤣💪,194,3611,2022-10-26 17:18:30,
7,@mtaibbi 🤣,117,2848,2022-10-26 17:14:07,
8,@ARodTV Definitely closer to citizen journalis...,699,10189,2022-10-26 17:05:16,Definitely closer to citizen journalism – loca...
9,@sandyleevincent Nobody bats 1000 🤷‍♂️,126,2920,2022-10-26 15:42:50,Nobody bats 1000


Some of the rows in the CleanedTweets is blank. So let's remove those rows

In [6]:
#remove the blank rows

tweet.drop(tweet[tweet['CleanedTweets']==''].index, inplace=True)

In [7]:
tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2668 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tweets         2668 non-null   object
 1   Retweets       2668 non-null   int64 
 2   Likes          2668 non-null   int64 
 3   Date           2668 non-null   object
 4   CleanedTweets  2668 non-null   object
dtypes: int64(2), object(3)
memory usage: 125.1+ KB


In [8]:
tweet['CleanedTweets'].head()

0                                            thanks
1                                        Absolutely
2                          Dear Twitter Advertisers
4    Meeting a lot of cool people at Twitter today!
5           Entering Twitter HQ – let that sink in!
Name: CleanedTweets, dtype: object

## Sensitivity Analysis of the tweets

In [9]:
#create a user define function to find out polarity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

#creating another user define function to label the sentiment 

def getSentiment(value):
    if value<0:
        return 'Negative'
    elif value>0:
        return 'Positive'
    else:
        return 'Neutral'

In [10]:
tweet['Polarity']=tweet['CleanedTweets'].apply(getPolarity)
tweet['Sentiment']=tweet['Polarity'].apply(getSentiment)

tweet.head(10)

,Tweets,Retweets,Likes,Date,CleanedTweets,Polarity,Sentiment
0,@PeterSchiff 🤣 thanks,209,7021,2022-10-27 16:17:39,thanks,0.200000,Positive
1,@ZubyMusic Absolutely,755,26737,2022-10-27 13:19:25,Absolutely,0.200000,Positive
2,Dear Twitter Advertisers https://t.co/GMwHmInPAS,55927,356623,2022-10-27 13:08:00,Dear Twitter Advertisers,0.000000,Neutral
4,Meeting a lot of cool people at Twitter today!,9366,195546,2022-10-26 21:39:32,Meeting a lot of cool people at Twitter today!,0.437500,Positive
5,Entering Twitter HQ – let that sink in! https:...,145520,1043592,2022-10-26 18:45:58,Entering Twitter HQ – let that sink in!,0.000000,Neutral
8,@ARodTV Definitely closer to citizen journalis...,699,10189,2022-10-26 17:05:16,Definitely closer to citizen journalism – loca...,0.166667,Positive
9,@sandyleevincent Nobody bats 1000 🤷‍♂️,126,2920,2022-10-26 15:42:50,Nobody bats 1000,0.000000,Neutral
10,A beautiful thing about Twitter is how it empo...,37951,294406,2022-10-26 15:27:40,A beautiful thing about Twitter is how it empo...,0.675000,Positive
11,@teslaownersSV I’m a big fan of citizen journa...,488,5529,2022-10-26 15:22:43,I’m a big fan of citizen journalism!,0.000000,Neutral
12,“According to unnamed sources close to the mat...,4603,62693,2022-10-26 14:50:58,“According to unnamed sources close to the mat...,0.000000,Neutral
